# Final Project

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

In [14]:
paths = list(Path("Smart Building").rglob("*.*"))

co2_paths = [path for path in paths if path.name == 'co2.csv']
humidity_paths = [path for path in paths if path.name == 'humidity.csv']
temperature_paths = [path for path in paths if path.name == 'temperature.csv']
light_paths = [path for path in paths if path.name == 'light.csv']
pir_paths = [path for path in paths if path.name == 'pir.csv']

In [15]:
dataset = []
for light_path, temperature_path, co2_path, pir_path, humidity_path in zip(light_paths, temperature_paths, co2_paths, pir_paths, humidity_paths):
    light_df = pd.read_csv(light_path, names=['unix_time', 'light'], index_col='unix_time')
    temperature_df = pd.read_csv(temperature_path, names=['unix_time', 'temperature'], index_col='unix_time')
    co2_df = pd.read_csv(co2_path, names=['unix_time', 'co2'], index_col='unix_time')
    humidity_df = pd.read_csv(humidity_path, names=['unix_time', 'humidity'], index_col='unix_time')
    pir_df = pd.read_csv(pir_path, names=['unix_time', 'pir'], index_col='unix_time')
    
    df = pd.concat([light_df, temperature_df, co2_df, pir_df, humidity_df], axis=1)
    df['room'] = light_path.parent.name
    dataset.append(df)
data = pd.concat(dataset)

data.head()

,light,temperature,co2,pir,humidity,room
unix_time,,,,,,
1377299107,97.0,23.93,488.0,NaN,45.34,413
1377299108,97.0,23.94,495.0,0.0,45.34,413
1377299126,97.0,23.93,NaN,0.0,45.34,413
1377299131,97.0,23.93,NaN,NaN,45.31,413
1377299136,97.0,23.92,NaN,NaN,45.34,413


In [16]:
print("Percentage of data missing")
data.isnull().sum().sort_values(ascending = False)/len(data)*100

Percentage of data missing


pir            75.010129
light          54.306578
humidity       54.306564
temperature    54.306286
co2            54.288882
room            0.000000
dtype: float64

In [17]:
data = data.dropna()
data.head(10)

,light,temperature,co2,pir,humidity,room
unix_time,,,,,,
1377299108,97.0,23.94,495.0,0.0,45.34,413
1377457084,103.0,23.48,459.0,0.0,52.75,413
1377457963,104.0,23.50,456.0,0.0,52.71,413
1377462175,105.0,23.54,446.0,0.0,52.43,413
1377464765,111.0,23.56,463.0,0.0,52.30,413
1377467333,159.0,23.65,458.0,0.0,52.21,413
1377470795,162.0,23.93,473.0,0.0,51.76,413
1377475133,182.0,24.36,455.0,0.0,50.83,413
1377478615,225.0,24.75,456.0,0.0,49.87,413


In [18]:
data.shape

(135386, 6)

In [19]:
print("Percentage of data missing")
data.isnull().sum().sort_values(ascending = False)/len(data)*100

Percentage of data missing


light          0.0
temperature    0.0
co2            0.0
pir            0.0
humidity       0.0
room           0.0
dtype: float64

In [30]:
data["pir"] = data["pir"].apply(lambda x: 0 if x == 0 else 1)

data["pir"].describe()

count    135386.000000
mean          0.067193
std           0.250357
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: pir, dtype: float64

In [38]:
from collections import Counter


def detectOutliers(data, n, features):
    outlier_rows = []
    for col in features:
        Q1 = np.percentile(data[col], 25)
        Q3 = np.percentile(data[col], 75)
        IQR = Q3 - Q1
        outlierStep = IQR * 1.5
        outlierList = data[(data[col] < Q1 - outlierStep ) | (data[col] > Q3 + outlierStep)].index
        outlier_rows.extend(outlierList)
    # Counts the frequency of occurrences of each row index
    outlier_rows = Counter(outlier_rows)
    multiple_outliers = list(key for key, value in outlier_rows.items() if value > n)
    return multiple_outliers

outliers_to_drop = detectOutliers(data, 5, ["temperature", "light", "humidity", "co2"])
print("these rows will be dropped due to their outlier values: {} ".format(outliers_to_drop))

these rows will be dropped due to their outlier values: [1377565933, 1377566749, 1377569232, 1377570100, 1377570980, 1377571831, 1377825332, 1377826051, 1377826790, 1377828271, 1377829722, 1377907298, 1377908778, 1377911741, 1377912482, 1377914707, 1377468186, 1377476867, 1377549110, 1377792765, 1377792784, 1377793026, 1377819493, 1377882264, 1377894165, 1377938996, 1377941956, 1377565101, 1377446023, 1377457084, 1377457963, 1377466494, 1377467333, 1377470795, 1377472562, 1377474292, 1377475133, 1377478615, 1377481225, 1377517891, 1377598134, 1377602415, 1377603263, 1377605765, 1377940477, 1377942706, 1377944234, 1377944985, 1377946426, 1377947146, 1377948619, 1377949374, 1377950096, 1377952225, 1377952964, 1377955245, 1377955947, 1377956605, 1377958033, 1377958773, 1377961742, 1377962473, 1377963195, 1377964586, 1377965997, 1377967463, 1377970451, 1377479518, 1377480371, 1377482891, 1377564289, 1377568425, 1377822463, 1377908023, 1377910263, 1377910985, 1377913194, 1377913916, 1377462

In [39]:
print("Before removing the outliers we have {} rows".format(len(data)))
data = data.drop(outliers_to_drop, axis=0).reset_index(drop=True)
print("After removing the outliers we have {} rows".format(len(data)))

Before removing the outliers we have 135386 rows
After removing the outliers we have 128262 rows


In [27]:
import datetime


datetime = datetime.datetime.fromtimestamp(1377299108)
datetime

datetime.datetime(2013, 8, 24, 1, 5, 8)